In [7]:
import os
import re
import pandas as pd
from docx import Document

def extract_text_from_docx(file_path):
    """Извлекает текст из DOCX-файла."""
    try:
        doc = Document(file_path)
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        return '\n'.join(full_text)
    except Exception as e:
        print(f"❌ Ошибка при извлечении текста: {e}")
        return None

def search_words_in_chapter(chapter_text, words):
    """Ищет слова в тексте главы и возвращает их с позициями."""
    word_pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    matches = []

    for match in re.finditer(word_pattern, chapter_text, re.IGNORECASE):
        start, end = match.start(), match.end()

        # Собираем контексты вокруг найденного слова
        start_context = chapter_text[:start].split()[-8:]
        end_context = chapter_text[end:].split()[:8]
        found_word = chapter_text[start:end]
        result = ' '.join(start_context + [found_word] + end_context)

        # Сохраняем позицию и результат
        matches.append((start, result))

    return matches

def search_in_all_chapters(chapters, words, category):
    """Ищет слова во всех главах и подсчитывает общее количество совпадений."""
    total_matches = 0
    results = []

    for chapter_number, chapter_title, chapter_text in chapters:
        matches = search_words_in_chapter(chapter_text, words)
        total_matches += len(matches)

        for position, match in matches:
            results.append([chapter_number, chapter_title, position, match, category])

    print(f"✅ Категория '{category}': найдено {total_matches} совпадений.")
    return results

def split_text_into_chapters(book_text):
    """Разделяет текст на главы. Пример: разделение по слову 'Chapter'."""
    chapters = []
    chapter_texts = book_text.split('Chapter ')
    for i, text in enumerate(chapter_texts[1:], start=1):
        title_end = text.find('\n')
        chapter_title = text[:title_end].strip()
        chapter_content = text[title_end + 1:].strip()
        chapters.append((i, chapter_title, chapter_content))
    return chapters

def main(docx_file_path, chapters_folder):
    """Основная логика обработки и поиска."""
    print(f"📂 Проверка наличия файла: {docx_file_path}")
    if not os.path.exists(docx_file_path):
        print(f"❌ Файл не найден: {docx_file_path}")
        return

    print("📖 Извлечение текста из файла...")
    book_text = extract_text_from_docx(docx_file_path)
    if not book_text:
        print("❌ Не удалось извлечь текст из файла.")
        return

    print("✂️ Разделение текста на главы...")
    chapters = split_text_into_chapters(book_text)
    if not chapters:
        print("❌ Не удалось разделить текст на главы.")
        return

    # Списки ключевых слов для поиска
    clothes_words = ['stilettos', 'sunglasses', 'ring', 'necklace', 'bracelet', 'earrings', 'brooch', 'watch', 'anklet', 'choker', 'pendant', 'cufflinks', 'tie clip', 'nose ring', 'belly button ring', 'toe ring', 'hairpin', 'tiara', 'diadem', 'bangle', 'chain', 'medallion', 'pearl necklace', 'locket', 'armband', 'charm bracelet', 'dress', 'robe', 'suit', 'clothes', 'coat', 'jacket', 'shirt', 'pants', 'skirt', 'jeans', 't-shirt', 'sweater', 'blouse', 'shorts', 'hoodie', 'vest', 'scarf', 'hat', 'gloves', 'boots', 'shoes', 'sneakers', 'socks', 'tie', 'belt', 'gown', 'trench coat', 'blazer', 'cardigan', 'overalls', 'tank top', 'leggings']
    hair_words = ['hair', 'beard', 'ponytail', 'bun', 'braids', 'bob', 'pixie cut', 'long waves', 'curly hair', 'straight hair', 'afro', 'buzz cut', 'french twist', 'dreadlocks', 'fishtail braid', 'half-up half-down', 'side part', 'middle part', 'updo', 'loose curls', 'locks', 'layered cut', 'shag cut', 'crew cut', 'mohawk', 'bangs', 'chignon', 'top knot']
    appearances_words = ['fur', 'black wolf', 'white wolf', 'brown wolf', 'caramel hair', 'blonde', 'brunette', 'redhead', 'white hair', 'red hair', 'auburn hair', 'chestnut hair', 'black hair', 'grey hair', 'dark hair', 'blue eyes', 'blue irises', 'blue eyeballs', 'brown eyes', 'brown irises', 'brown eyeballs', 'black eyes', 'black irises', 'black eyeballs', 'red eyes', 'red irises', 'red eyeballs', 'hazel eyes', 'hazel irises', 'hazel eyeballs', 'green eyes', 'green irises', 'green eyeballs', 'eyes were green', 'eyes were black', 'eyes were brown', 'eyes were hazel', 'eyes were blue', 'eyes were grey', '5 feet', "5'", "6'", "7'", '6 feet', '7 feet', 'feet tall', 'slim', 'thin', 'thick', 'tall', 'dark skin', 'white skin', 'pale skin', 'freckles', 'tattoos', 'tattoo', 'brown skin', 'black skin', 'high cheekbones', 'wrinkles', 'wrinkled', 'full lips', 'small breasts', 'curves', 'big breasts']
    other_words = ['eyes', 'face', 'skin', 'body', 'fur', 'chin', 'cheeks', 'big wolf', 'white wolf', 'beautiful', 'ugly', 'handsome', 'cute', 'gorgeous', 'sharp', 'features', 'arms', 'bicep', 'legs', 'ass', 'breasts', 'waist', 'muscular', 'pale', 'features', 'thin', 'fangs', 'tattoos', 'teeth', 'mouth', 'young', 'old', 'blood', 'bleeding', 'gaze', 'smirk', 'smile', 'lips', 'nose', 'hands', 'jaw']
    weather_words = ['morning', 'afternoon', 'evening', 'night', 'sunrise', 'sunset', 'dawn', 'dusk', 'noon', 'midnight', 'cloudy', 'rain', 'storm', 'wind', 'sun', 'sunny', 'fog', 'foggy', 'snow', 'snowy', 'hail', 'thunder', 'lightning', 'breeze', 'chilly', 'hot', 'warm', 'cold', 'frost', 'blizzard', 'temperature', 'humid', 'dry', 'drizzle', 'pouring', 'downpour', 'mist', 'overcast']
    locations_words = ['forest', 'living room', 'dining room', 'school', 'college', 'training grounds', 'field', 'bathroom', 'bedroom', 'cabin', 'house']
    age_words = ['sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'twenties', 'thirties', 'years old', 'forties', 'fifties', 'sixties', 'seventies']
   

    all_results = []
    for words, category in [
        (clothes_words, 'Clothes'), (hair_words, 'Hair'),
        (appearances_words, 'Appearances'), (weather_words, 'Weather'),
        (locations_words, 'Locations'), (age_words, 'Age'), (other_words, 'Other')
    ]:
        all_results.extend(search_in_all_chapters(chapters, words, category))

    print("📊 Создание DataFrame с результатами...")
    all_data_df = pd.DataFrame(all_results, columns=['Chapter Number', 'Chapter Title', 'Position', 'Match', 'Category'])
    
    # Сортируем по номеру главы и позиции
    all_data_df.sort_values(by=['Chapter Number', 'Position'], inplace=True)

    print("💾 Сохранение результатов в Excel...")
    os.makedirs(chapters_folder, exist_ok=True)
    excel_file_path = f"{chapters_folder}/details.xlsx"
    
    # Убираем колонку 'Position' перед сохранением
    try:
        all_data_df.drop(columns=['Position']).to_excel(excel_file_path, sheet_name='Details', index=False)
        print(f"✅ Файл Excel создан: {excel_file_path}")
    except Exception as e:
        print(f"❌ Ошибка при сохранении Excel файла: {e}")

# Запуск основного процесса
chapters_folder = "daddy_where_have"
docx_file_path = f"{chapters_folder}/fulltext.docx"
main(docx_file_path, chapters_folder)


📂 Проверка наличия файла: daddy_where_have/fulltext.docx
📖 Извлечение текста из файла...
✂️ Разделение текста на главы...
❌ Не удалось разделить текст на главы.


In [ ]:
##Если проблема с делением на главы в первом скрипте, ищем по файлам отдельных глав##

In [10]:
import os
import re
import pandas as pd
from docx import Document

def extract_text_from_docx(file_path):
    """Извлекает текст из DOCX-файла."""
    try:
        doc = Document(file_path)
        full_text = []
        for paragraph in doc.paragraphs:
            full_text.append(paragraph.text)
        return '\n'.join(full_text)
    except Exception as e:
        print(f"❌ Ошибка при извлечении текста: {e}")
        return None

def load_chapters_from_directory(directory):
    """Загружает главы из файлов DOCX в указанной директории."""
    chapters = []
    for filename in sorted(os.listdir(directory)):
        if filename.startswith("Chapter_") and filename.endswith(".docx"):
            chapter_number = int(re.search(r'\d+', filename).group())  # Извлекаем номер главы
            chapter_title = f"Chapter {chapter_number}"
            file_path = os.path.join(directory, filename)
            print(f"📄 Обработка файла: {filename}")
            chapter_text = extract_text_from_docx(file_path)
            if chapter_text:
                chapters.append((chapter_number, chapter_title, chapter_text))
            else:
                print(f"❌ Не удалось загрузить текст из {filename}")
    return chapters

def search_words_in_chapter(chapter_text, words):
    """Ищет слова в тексте главы и возвращает их с позициями."""
    word_pattern = r'\b(' + '|'.join(re.escape(word) for word in words) + r')\b'
    matches = []

    for match in re.finditer(word_pattern, chapter_text, re.IGNORECASE):
        start, end = match.start(), match.end()

        # Собираем контексты вокруг найденного слова
        start_context = chapter_text[:start].split()[-8:]
        end_context = chapter_text[end:].split()[:8]
        found_word = chapter_text[start:end]
        result = ' '.join(start_context + [found_word] + end_context)

        # Сохраняем позицию и результат
        matches.append((start, result))

    return matches

def search_in_all_chapters(chapters, words, category):
    """Ищет слова во всех главах и подсчитывает общее количество совпадений."""
    total_matches = 0
    results = []

    for chapter_number, chapter_title, chapter_text in chapters:
        matches = search_words_in_chapter(chapter_text, words)
        total_matches += len(matches)

        for position, match in matches:
            results.append([chapter_number, chapter_title, position, match, category])

    print(f"✅ Категория '{category}': найдено {total_matches} совпадений.")
    return results

def main(chapters_folder):
    """Основная логика обработки и поиска."""
    print(f"📂 Проверка директории: {chapters_folder}")
    if not os.path.exists(chapters_folder):
        print(f"❌ Директория не найдена: {chapters_folder}")
        return

    print("📖 Загрузка глав из файлов...")
    chapters = load_chapters_from_directory(chapters_folder)
    if not chapters:
        print("❌ Не удалось найти главы в директории.")
        return

    # Списки ключевых слов для поиска (пример заполнения)
    clothes_words = ['stilettos', 'sunglasses', 'ring', 'necklace', 'bracelet', 'earrings', 'brooch', 'watch', 'anklet', 'choker', 'pendant', 'cufflinks', 'tie clip', 'nose ring', 'belly button ring', 'toe ring', 'hairpin', 'tiara', 'diadem', 'bangle', 'chain', 'medallion', 'pearl necklace', 'locket', 'armband', 'charm bracelet', 'dress', 'robe', 'suit', 'clothes', 'coat', 'jacket', 'shirt', 'pants', 'skirt', 'jeans', 't-shirt', 'sweater', 'blouse', 'shorts', 'hoodie', 'vest', 'scarf', 'hat', 'gloves', 'boots', 'shoes', 'sneakers', 'socks', 'tie', 'belt', 'gown', 'trench coat', 'blazer', 'cardigan', 'overalls', 'tank top', 'leggings']
    hair_words = ['hair', 'beard', 'ponytail', 'bun', 'braids', 'bob', 'pixie cut', 'long waves', 'curly hair', 'straight hair', 'afro', 'buzz cut', 'french twist', 'dreadlocks', 'fishtail braid', 'half-up half-down', 'side part', 'middle part', 'updo', 'loose curls', 'locks', 'layered cut', 'shag cut', 'crew cut', 'mohawk', 'bangs', 'chignon', 'top knot']
    appearances_words = ['fur', 'black wolf', 'white wolf', 'brown wolf', 'caramel hair', 'blonde', 'brunette', 'redhead', 'white hair', 'red hair', 'auburn hair', 'chestnut hair', 'black hair', 'grey hair', 'dark hair', 'blue eyes', 'blue irises', 'blue eyeballs', 'brown eyes', 'brown irises', 'brown eyeballs', 'black eyes', 'black irises', 'black eyeballs', 'red eyes', 'red irises', 'red eyeballs', 'hazel eyes', 'hazel irises', 'hazel eyeballs', 'green eyes', 'green irises', 'green eyeballs', 'eyes were green', 'eyes were black', 'eyes were brown', 'eyes were hazel', 'eyes were blue', 'eyes were grey', '5 feet', "5'", "6'", "7'", '6 feet', '7 feet', 'feet tall', 'slim', 'thin', 'thick', 'tall', 'dark skin', 'white skin', 'pale skin', 'freckles', 'tattoos', 'tattoo', 'brown skin', 'black skin', 'high cheekbones', 'wrinkles', 'wrinkled', 'full lips', 'small breasts', 'curves', 'big breasts']
    other_words = ['eyes', 'face', 'skin', 'body', 'fur', 'chin', 'cheeks', 'big wolf', 'white wolf', 'beautiful', 'ugly', 'handsome', 'cute', 'gorgeous', 'sharp', 'features', 'arms', 'bicep', 'legs', 'ass', 'breasts', 'waist', 'muscular', 'pale', 'features', 'thin', 'fangs', 'tattoos', 'teeth', 'mouth', 'young', 'old', 'blood', 'bleeding', 'gaze', 'smirk', 'smile', 'lips', 'nose', 'hands', 'jaw']
    weather_words = ['morning', 'afternoon', 'evening', 'night', 'sunrise', 'sunset', 'dawn', 'dusk', 'noon', 'midnight', 'cloudy', 'rain', 'storm', 'wind', 'sun', 'sunny', 'fog', 'foggy', 'snow', 'snowy', 'hail', 'thunder', 'lightning', 'breeze', 'chilly', 'hot', 'warm', 'cold', 'frost', 'blizzard', 'temperature', 'humid', 'dry', 'drizzle', 'pouring', 'downpour', 'mist', 'overcast']
    locations_words = ['forest', 'living room', 'dining room', 'school', 'college', 'training grounds', 'field', 'bathroom', 'bedroom', 'cabin', 'house']
    age_words = ['sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty', 'thirty', 'forty', 'fifty', 'twenties', 'thirties', 'years old', 'forties', 'fifties', 'sixties', 'seventies']
   
    all_results = []
    for words, category in [
        (clothes_words, 'Clothes'), (hair_words, 'Hair'),
        (appearances_words, 'Appearances'), (weather_words, 'Weather'),
        (locations_words, 'Locations'), (age_words, 'Age'), (other_words, 'Other')
    ]:
        all_results.extend(search_in_all_chapters(chapters, words, category))

    print("📊 Создание DataFrame с результатами...")
    all_data_df = pd.DataFrame(all_results, columns=['Chapter Number', 'Chapter Title', 'Position', 'Match', 'Category'])

    # Сортируем по номеру главы и позиции
    all_data_df.sort_values(by=['Chapter Number', 'Position'], inplace=True)

    print("💾 Сохранение результатов в Excel...")
    excel_file_path = os.path.join(chapters_folder, "details.xlsx")

    # Убираем колонку 'Position' перед сохранением
    try:
        all_data_df.drop(columns=['Position']).to_excel(excel_file_path, sheet_name='Details', index=False)
        print(f"✅ Файл Excel создан: {excel_file_path}")
    except Exception as e:
        print(f"❌ Ошибка при сохранении Excel файла: {e}")

# Запуск основного процесса
chapters_folder = "daddy_where_have"
main(chapters_folder)


📂 Проверка директории: daddy_where_have
📖 Загрузка глав из файлов...
📄 Обработка файла: Chapter_0.docx
📄 Обработка файла: Chapter_1.docx
📄 Обработка файла: Chapter_10.docx
📄 Обработка файла: Chapter_100.docx
📄 Обработка файла: Chapter_101.docx
📄 Обработка файла: Chapter_102.docx
📄 Обработка файла: Chapter_103.docx
📄 Обработка файла: Chapter_104.docx
📄 Обработка файла: Chapter_105.docx
📄 Обработка файла: Chapter_106.docx
📄 Обработка файла: Chapter_107.docx
📄 Обработка файла: Chapter_108.docx
📄 Обработка файла: Chapter_109.docx
📄 Обработка файла: Chapter_11.docx
📄 Обработка файла: Chapter_110.docx
📄 Обработка файла: Chapter_111.docx
📄 Обработка файла: Chapter_112.docx
📄 Обработка файла: Chapter_113.docx
📄 Обработка файла: Chapter_114.docx
📄 Обработка файла: Chapter_115.docx
📄 Обработка файла: Chapter_116.docx
📄 Обработка файла: Chapter_117.docx
📄 Обработка файла: Chapter_118.docx
📄 Обработка файла: Chapter_119.docx
📄 Обработка файла: Chapter_12.docx
📄 Обработка файла: Chapter_120.docx
📄 